# **INTRODUCTIONS**

---

The **boostershots** are additional vaccination shot when the effectiveness - regarding the first series of vaccinations - has been reduced and the **third vaccincation shot** are additional vaccination shot for immune sensitive people. Both vaccinations are represented on the **[vaccination page](https://coronadashboard.rijksoverheid.nl/landelijk/vaccinaties)**.   

The source data (i.e. **[RIVM](https://www.rivm.nl/en)**) are modelled on daily registrations.

The code is separated into multiple sections:

1. **[Views](#views)**
2. **[Datatino Configurations](#datatino-configurations)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/dataflows/rivm_booster_shots_administered.ipynb",
        "src/dataflows/rivm_third_shots_administered.ipynb",
        "src/common/schemas.ipynb"
    ]
}
```

# **VIEWS**
---

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSDEST].[V_RIVM_VACCINATIONS_BOOSTER_AND_THIRD_SHOTS_ADMINISTERED_NL]
AS
    WITH ADMINISTERED_CTE AS (
        SELECT
            'THIRD_SHOTS' AS [TYPE_SHOT],
            [ADMINISTERED_TOTAL],
            [DATE_OF_INSERTION_UNIX],
            [DATE_UNIX]
        FROM [VWSDEST].[V_RIVM_VACCINATIONS_THIRD_SHOTS_ADMINISTERED_NL]
        UNION
        SELECT
            'BOOSTER_SHOTS' AS [TYPE_SHOT],
            [GGD_ADMINISTERED_TOTAL] + [OTHERS_ADMINISTERED_TOTAL],
            [DATE_OF_INSERTION_UNIX],
            [DATE_END_UNIX]
        FROM [VWSDEST].[V_RIVM_VACCINATIONS_BOOSTER_SHOTS_ADMINISTERED_NL]
    )
    SELECT 
        SUM([ADMINISTERED_TOTAL])  AS [ADMINISTERED_TOTAL],
        MAX([DATE_OF_INSERTION_UNIX]) AS [DATE_OF_INSERTION_UNIX],
        MAX([DATE_UNIX]) AS [DATE_UNIX]
    FROM ADMINISTERED_CTE
GO


# **DATATINO CONFIGURATIONS**
---

### PROTOS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @config_id INT, 
        @view_id INT,
        @view_name VARCHAR(256) = 'VWSDEST.V_RIVM_VACCINATIONS_BOOSTER_AND_THIRD_SHOTS_ADMINISTERED_NL',
        @item_name VARCHAR(256) = 'booster_and_third_shot_administered',
        @view_description VARCHAR(256),
        @config_description VARCHAR(256),
        @proto_name VARCHAR(50) = 'NL',
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name);

SELECT TOP(1)
    @config_id = configs.[ID],
    @view_id = views.[ID]
FROM [DATATINO_PROTO_1].[CONFIGURATIONS] configs
INNER JOIN [DATATINO_PROTO_1].[VIEWS] views ON views.[ID] = configs.[VIEW_ID] AND views.[NAME] = @view_name
INNER JOIN [DATATINO_PROTO_1].[PROTOS] protos ON protos.[ID] = configs.[PROTO_ID] AND protos.[NAME] = @proto_name
WHERE configs.[NAME] = @item_name;

-- 2) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = 'DATE_UNIX',
    @constraint_key_name = null,
    @constraint_value = null,
    @grouped_key_name = null,
    @grouped_last_update_name = null;

-- 3) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = '0',
    @grouped = '0',
    @columns = '*',
    @mapping = '=LOWER()',
    @layout_type_id = '1',
    @active = @is_active;
GO